<a href="https://colab.research.google.com/github/bata0701/BERT_Japanese_Google_Colaboratory/blob/master/2_BERT_livedoor_news_on_Google_Colaboratory_NEW_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最新でも稼働するように修正中

## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2023-03-12 07:36:35--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.80MB/s    in 6.7s    

2023-03-12 07:36:44 (1.26 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['smax', 'peachy', 'movie-enter', 'CHANGES.txt', 'kaden-channel', 'README.txt', 'dokujo-tsushin', 'topic-news', 'livedoor-homme', 'it-life-hack', 'sports-watch']
カテゴリー数: 9
['smax', 'peachy', 'movie-enter', 'kaden-channel', 'dokujo-tsushin', 'topic-news', 'livedoor-homme', 'it-life-hack', 'sports-watch']


In [5]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [6]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [7]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [8]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


AQUOS PHONE Xx 106SHにソフトウェア更新！ソフトバンクモバイルは21日、今夏モデルのAndroid 4.0（開発コード名：IceCream Sandwich；ICS ）を搭載したスマートフォン「AQUOS PHONE Xx 106SH」（シャープ製）においてブラウザが強制終了する不具合が見つかったとしてネットワーク経由による本体ファームウェアアップデートサービス「ソフトウェア更新」を提供開始したことをお知らせしています。今回の更新で修正される不具合は以下の1点のみです。更新にかかる時間は最大20分程度、更新期間は3年となります。ただし、更新期間は予告なく延長することがあるということです。AQUOS PHONE Xx 106SHのソフトウェア更新は今回がはじめて。ブラウザが強制終了する場合がある。※  上記事象以外にも快適にご利用いただくための更新が含まれておりますソフトウェア更新の手順は「ソフトウェア更新手順（PDF版）」を参照してください。以下、注意点などです。ネットワークを利用したソフトウェア更新を実施させていただきます。・対象のお客様には順次、お知らせメール（SMS）をお送りいたします。・お知らせメールに記載の予定日時に自動ダウンロードが開始されます。（自動ダウンロードの時間変更・取消はできません）・自動ダウンロードが完了すると、画面上部にマークが表示されますので、お客さまにてソフトウェア更新をお願いいたします。※  自動ダウンロード完了後、あらかじめ指定された時刻(初期設定は午前3時)にソフトウェア更新が行われます。・自動ダウンロードを待たずに、お客さま自身で即時ソフトウェア更新を実施して頂くことも可能です。※  ソフトウェア更新の具体的な操作方法につきましては、上記のソフトウェア更新手順をご参照ください。※  ソフトウェアのダウンロードは3G回線/Wi-Fi回線のどちらでも実施可能です。<<バージョン確認手順>>【設定】→【端末情報】→【ビルド番号】<<最新のソフトウェアバージョン（ビルド番号）>>【106SH】（ビルド番号）：S0020＜ソフトウェア更新を実施いただく上での注意点＞●ソフトウェア書き換え中は、発着信を含む携帯電話の各機能をご利用できません。また、緊急通報（110番、118番、119番）をご利用することもできません。 一回

In [9]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,AQUOS PHONE Xx 106SHにソフトウェア更新！ソフトバンクモバイルは21日、今...,smax
1,2012年7月16〜22日に紹介したAndroidアプリ！先週はNTTドコモ向け「ARROW...,smax
2,ゼロから始めるスマートフォンまだ詳細なスペックが発表されていない「Xperia GX」と「X...,smax
3,イー・モバイルの2012年夏モデル発表会！イー・アクセスは6日、イー・モバイル向けに今夏以降...,smax
4,発売前のAQUOS PHONE si SH-01Eを体験！28日に発表したNTTドコモ「20...,smax


In [10]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'smax', 1: 'peachy', 2: 'movie-enter', 3: 'kaden-channel', 4: 'dokujo-tsushin', 5: 'topic-news', 6: 'livedoor-homme', 7: 'it-life-hack', 8: 'sports-watch'}
{'smax': 0, 'peachy': 1, 'movie-enter': 2, 'kaden-channel': 3, 'dokujo-tsushin': 4, 'topic-news': 5, 'livedoor-homme': 6, 'it-life-hack': 7, 'sports-watch': 8}


,text,label_index
0,AQUOS PHONE Xx 106SHにソフトウェア更新！ソフトバンクモバイルは21日、今...,0
1,2012年7月16〜22日に紹介したAndroidアプリ！先週はNTTドコモ向け「ARROW...,0
2,ゼロから始めるスマートフォンまだ詳細なスペックが発表されていない「Xperia GX」と「X...,0
3,イー・モバイルの2012年夏モデル発表会！イー・アクセスは6日、イー・モバイル向けに今夏以降...,0
4,発売前のAQUOS PHONE si SH-01Eを体験！28日に発表したNTTドコモ「20...,0


In [11]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,18日、日本経済新聞が「被災地もう一つの異常事態 復興特需・原発賠償金・・・マネー流入ゆがむ...,5
1,前回の記事では、今までに自分がブックマークした記事や、他の人がブックマークした記事を探す「検...,6
2,日本国内では、今までのソフトバンク独占ではなく、auからもiPhone4Sが登場したことによ...,3
3,ドイツのサッカーリーグ、ブンデスリーガのバイエルン・ミュンヘンに所属する宇佐美貴史の妻でキャ...,5
4,就職氷河期の再来と言われる今年の就活環境、銀行や商社など100社を受けたがすべて書類選考で落...,4


In [12]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [13]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [25]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
# 2.9.0は上手く行かない
# !pip install transformers==2.9.0
# 3.5.1以降の場合は、BertModelの位置が変わっているので注意
!pip install transformers

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.12-1ubuntu4).
swig is already the newest version (4.0.1-5build1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
mecab is already installed at the requested version (0.996-10build1)
libmecab-dev is already installed at the requested version (0.996-10build1)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-2.1)
git is already installed at the requested version (1:2.25.1-1ubuntu3.10)
make is already installed at the requested version (4.2.1-1.2)
curl is already installed at the requested version (7.68.0-1ubuntu2.16)
xz-utils is already installed at the requested version (5.2.4-1ubuntu1.1)
file is already installed at the requested version (1:5.38-4)
mecab is already installed at the requested version (0.996-10build1)
libmecab-dev is already installed at the requested version (0.996-10build1)
mecab-

In [15]:
## 修正 ##
# 辞書をダウンロード
# これをしておかないと "no such file or directory: /xxx/xxx/xxx/mecabrc" が出る
# !pip install unidic-lite
!pip install unidic
!python -m unidic download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=7465cc8f11ef478cb83dc717d04dafb60e0a4ad2b0f4b4a73dd7edff16ddf7ba
  Stored in directory: /root/.cache/pip/wheels/69/2e/59/f3af29b54acb46ba8eebeb46e900a406dcd9ce00fb513ec3c1
Successfully built unidic
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:42<00:00, 12.4MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.9/dist-packages/unidic/dicdir


In [27]:
import torch
import torchtext  # torchtextを使用
# from transformers.modeling_bert import BertModel
# from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
# 上記が使えないので、モデル自体を変える

from transformers import BertModel
from transformers import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
#tokenizer = BertJapaneseTokenizer.from_pretrained(
#    'bert-base-japanese-whole-word-masking')

# 上記が上手く動かないので
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
## 修正 ##
# torchtextのdata（TabularDatasetなど）が使用できなくなったので

from torchtext.data.utils import get_tokenizer
# tokenizer設定
# 日本語の設定がわからないので、事前に分かち書きをしておく
tokenizer = get_tokenizer(tokenizer=None)

In [29]:
## 修正 ##
# 単語分割
import MeCab
import unidic

In [30]:
## 修正 ##
# リスト化されているtext列を結合

# txt_col = df.columns.get_loc('text')
# for i in range(len(df)):
#     temp = ",".join(df.iloc[i, txt_col])
#     df.iloc[i, txt_col] = temp

In [31]:
# df.iloc[0][txt_col]

In [32]:
## 修正 ##
# text列を分かち書き
wakati = MeCab.Tagger('-Owakati')

txt_col = df.columns.get_loc('text')

text_list = []
for i in range(len(df)):
    result = wakati.parse(df.iloc[i, txt_col])
    result = result.replace(' ', ',')
    df.iloc[i, txt_col] = result

In [33]:
df.iloc[0][txt_col]

'18,日,、,日本,経済,新聞,が,「,被災,地,もう,一,つ,の,異常,事態,復興,特需,・,原発,賠償,金,・,・,・,マネー,流入,ゆがむ,再生,」,と,題し,た,記事,で,被災,地,の,歪ん,だ,現実,を,報じ,、,ネット,掲示,板,で,物議,を,醸し,て,いる,。,同,記事,で,は,、,賠償,を,受ける,対象,の,家族,が,五,人,家族,で,あれ,ば,月,80,万,円,が,懐,に,入る,と,指摘,し,、,「,東電,から,金,もらっ,て,、,働か,なく,て,も,パチンコ,し,たり,、,すし,食っ,たり,スマホ,と,か,ばんばん,新しく,し,て,いる,」,と,いう,現地,の,人,の,言葉,を,掲載,。,この,被災,者,に,流れる,賠償,金,と,その,使い,方,を,「,ゆがむ,再生,」,と,し,て,位置,づけ,て,いる,。,ネット,掲示,板,で,は,「,一切,募金,し,なかっ,た,おれ,に,死角,は,なかっ,た,な,」,「,震災,さまさま,や,な,」,「,スマホ,と,寿司,は,消費,だ,から,かまわ,ん,の,だ,が,、,パチンコ,は,ダメ,だ,って,・,・,・,。,ほんと,田舎,は,感覚,が,狂っ,てる,」,「,で,、,何,年,ぐらい,もらえる,ん,だ,？,5,年,？,5,年,で,月,80,万,なら,4800,万,だ,ぞ,」,「,ただ,の,クズ,ニート,じゃん,」,「,被災,者,だ,から,って,特別,扱い,し,過ぎ,てる,だろ,」,など,、,東電,から,賠償,金,を,貰っ,て,余裕,の,ある,暮らし,を,し,て,いる,被災,者,を,糾弾,する,声,が,殺到,し,て,いる,。,一方,で,「,土地,と,家,と,家,の,中,の,物,全て,失っ,てる,時点,で,ン,千,万,以上,パー,に,なっ,てる,じゃん,。,この,カネ,も,いつ,まで,もらえる,の,か,も,わから,ん,し,」,「,じゃあ,お前,も,、,その,金,を,貰う,ため,に,人生,の,全て,を,失っ,て,みる,か,？,」,など,、,安易,な,被災,者,批判,を,非難,する,声,も,多く,見,られ,た,。,ほか,に,も,「,家族,や,財産,が,自分,以外,全部,流さ,れ,たら,毎日,パチンコ,や,酒,に,逃げる,の,は,わかる,。,実際,パチンコ,行く,人,は,そう,いう,人,が,多い,ん,だろ,」,と,、,

In [34]:
## 修正 ##
# 単語辞書作成
from torchtext.vocab import build_vocab_from_iterator

# 単語辞書作成
text_vocab = build_vocab_from_iterator(df['text'], specials=('<unk>', '<pad>'))
text_vocab.set_default_index(text_vocab['<unk>'])

In [35]:
## 修正 ##
# 単語内容確認
text_vocab.get_stoi()

{'＄': 3790,
 '\ufeff': 3789,
 '\ue2c0': 3787,
 '\ue239': 3786,
 '혐': 3783,
 '풍': 3782,
 '틀': 3779,
 '크': 3777,
 '치': 3774,
 '차': 3770,
 '지': 3769,
 '중': 3767,
 '죠': 3765,
 '져': 3764,
 '유': 3760,
 '위': 3758,
 '요': 3756,
 '옆': 3753,
 '엄': 3752,
 '알': 3751,
 '악': 3750,
 '속': 3746,
 '세': 3745,
 '생': 3743,
 '삼': 3741,
 '빠': 3740,
 '뷰': 3738,
 '별': 3736,
 '벌': 3735,
 '막': 3729,
 '론': 3728,
 '략': 3725,
 '들': 3722,
 '둑': 3720,
 '동': 3719,
 '돌': 3718,
 '노': 3714,
 '네': 3713,
 '근': 3706,
 '광': 3705,
 '거': 3703,
 '개': 3701,
 '齎': 3698,
 '麿': 3697,
 '鹹': 3696,
 '鷺': 3695,
 '鮪': 3691,
 '魏': 3689,
 '驟': 3682,
 '驕': 3681,
 '餡': 3678,
 '餞': 3677,
 '飄': 3676,
 '霹': 3673,
 '雙': 3671,
 '鏈': 3668,
 '鎬': 3666,
 '錨': 3665,
 '錐': 3664,
 '鉾': 3663,
 '鉉': 3661,
 '躙': 3657,
 '躁': 3656,
 '蹊': 3654,
 '蹄': 3653,
 '蹂': 3652,
 '饗': 3680,
 '謂': 3649,
 '謁': 3648,
 '褻': 3644,
 '褥': 3643,
 '褄': 3642,
 '蟄': 3640,
 '蝙': 3638,
 '蜃': 3637,
 '蛎': 3635,
 '蚕': 3633,
 '藪': 3632,
 '蕉': 3627,
 '蓑': 3625,
 '蒜': 3622,
 '菟': 3617,
 

In [36]:
df

,text,label_index
0,"18,日,、,日本,経済,新聞,が,「,被災,地,もう,一,つ,の,異常,事態,復興,特需,...",5
1,"前回,の,記事,で,は,、,今,まで,に,自分,が,ブックマーク,し,た,記事,や,、,他,...",6
2,"日本,国,内,で,は,、,今,まで,の,ソフト,バンク,独占,で,は,なく,、,au,から,...",3
3,"ドイツ,の,サッカー,リーグ,、,ブンデスリーガ,の,バイエルン,・,ミュンヘン,に,所属,...",5
4,"就職,氷河,期,の,再来,と,言わ,れる,今年,の,就活,環境,、,銀行,や,商社,など,1...",4
...,...,...
7371,"『,日本,レース,クイーン,大賞,』,の,最終,選考,が,東京,オート,サロン,で,行なわ,...",6
7372,"ここ,数,年,、,美容,整形,に,対する,意識,は,、,その,手軽,さ,から,だいぶ,変わっ...",4
7373,"彼,の,部屋,に,お,呼ばれ,する,と,、,彼,の,部屋,を,チェック,し,て,しまう,の,...",1
7374,"昨年,の,秋,、,希望,の,職種,に,転職,し,た,カナコ,さん,（,30,歳,／,商社,勤...",4


In [37]:
## 修正 ##
# ラベル辞書
df['label_index'] = df['label_index'].astype(str)  # ラベルが数値のため文字型に変換。文字型で指定していた場合はこの処理は不要
label_vocab = build_vocab_from_iterator(df['label_index'])

In [38]:
## 修正 ##
label_vocab.get_stoi()

{'6': 8, '5': 7, '1': 6, '3': 5, '7': 4, '4': 3, '2': 2, '0': 1, '8': 0}

In [39]:
## 修正 ##
import torchtext.transforms as T

# transform生成
text_transform = T.Sequential(
    T.VocabTransform(text_vocab),
    T.ToTensor(padding_value=text_vocab['<pad>'])
)
label_transform = T.Sequential(
    T.VocabTransform(label_vocab),
    T.ToTensor()
)

In [43]:
## 修正 ##
# ミニバッチ時のデータ変換関数
def collate_batch(batch):
    texts = text_transform([text for (text, label_index) in batch])
    labels = label_transform([label for (text, label_index) in batch])
    return texts, labels

In [44]:
## 修正 ##
from torch.utils.data import DataLoader

# DataLoader設定
data_loader = DataLoader(df.values, batch_size=3, shuffle=True, collate_fn=collate_batch)

# ここからエラー

In [45]:
## 修正 ##
# 各バッチごとのデータを確認
for i, (texts, labels) in enumerate(data_loader):
    print(i)
    for text, label in zip(texts, labels):
        print(text, label_vocab.lookup_token(label))

RuntimeError: ignored

In [42]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


AttributeError: ignored

In [ ]:
for i, (texts, labels) in enumerate(data_loader):
    print(i)
    for text, label in zip(texts, labels):
        print(text, label_vocab.lookup_token(label))

In [ ]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

In [ ]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [ ]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


In [ ]:
# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)


In [ ]:
# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章
dic_id2cat[int(item.Label)]  # id


In [ ]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)

## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [ ]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
print(model)


In [ ]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output


In [ ]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


## 準備4：BERTのファインチューニングの設定

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [ ]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## 5. 訓練を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [ ]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


## テストデータでの性能を確認

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


https://yoheikikuta.github.io/bert-japanese/

https://github.com/yoheikikuta/bert-japanese

の「BERT with SentencePiece for Japanese text.」

では、入力テキストにタイトルを含めていますが、今回はタイトルは除いています。

同様にタイトルを抜いている、[BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)でも、正解率が92%ちょっととなっており、ほぼ同じ正解率が得られました。

以上。